In [2]:
import cv2

# start up the webcam
cap = cv2.VideoCapture(1)

# check if the webcam can be accessed
if not cap.isOpened():
    raise IOError("Cannot open webcam")

#take one frame from the webcam 
ret, frame = cap.read()

# check if a frame has been captured successfully
if ret:
    # save the frame as an image file
    cv2.imwrite('webcam_image.jpg', frame)
else:
    print("Error: No frame captured")

# close the webcam
cap.release()
# close all openCV functions
cv2.destroyAllWindows()

In [3]:
# start up the webcam
cap = cv2.VideoCapture(1)

# initialise the codec and the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

# check if the webcam can be accessed
if not cap.isOpened():
    raise IOError("Cannot open webcam")

try:
    while True:
        # capture frame by frame from the webcam
        ret, frame = cap.read()

        if not ret:
            break

        # write the frame on to a file using the "out" video writer
        out.write(frame)

# stop the recording on an interrupt
except KeyboardInterrupt:
    print("Stream stopped")
    
finally:
    # close webcam and all openCV functions
    cap.release()
    out.release()
    cv2.destroyAllWindows()

Stream stopped


In [ ]:
import cv2
import mediapipe as mp

# initialise the codec and the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

#initialise media pipe's face detection and video drawing 
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection()

cap = cv2.VideoCapture(1)

# check if the webcam can be accessed
if not cap.isOpened():
    raise IOError("Cannot open webcam")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # convert captured frame to colour
        frame_colour = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # apply the face detection
        results = face_detection.process(frame_colour)

        # draw the face detection mapping onto the frame
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)

        # write the frame on to a file using the "out" video writer
        out.write(frame)


except KeyboardInterrupt:
    print("Stream stopped")
    
finally:
    # close webcam and all openCV functions
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [4]:
def get_head_pose(landmarks, image_shape):
    # creates a 3d coordinate system model of a face
    model_points = np.array([
        (0.0, 0.0, 0.0),             # nose tip
        (0.0, -330.0, -65.0),        # chin
        (-225.0, 170.0, -135.0),     # left eye left corner
        (225.0, 170.0, -135.0),      # right eye right corner
        (-150.0, -150.0, -125.0),    # left Mouth corner
        (150.0, -150.0, -125.0)      # right mouth corner
    ])
    
    # using media pipe face gets the point corresponding to the model face
    image_points = np.array([
        (landmarks[1].x * image_shape[1], landmarks[1].y * image_shape[0]),  # nose tip
        (landmarks[152].x * image_shape[1], landmarks[152].y * image_shape[0]),  # chin
        (landmarks[226].x * image_shape[1], landmarks[226].y * image_shape[0]),  # left eye left corner
        (landmarks[446].x * image_shape[1], landmarks[446].y * image_shape[0]),  # right eye right corner
        (landmarks[57].x * image_shape[1], landmarks[57].y * image_shape[0]),  # left Mouth corner
        (landmarks[287].x * image_shape[1], landmarks[287].y * image_shape[0])  # right mouth corner
    ], dtype="double")

    # calculates the camera matrix
    principal_point = (image_shape[1]/2, image_shape[0]/2)
    focal_length = principal_point[0] / np.tan(60 / 2 * np.pi / 180)
    camera_matrix = np.array(
        [[focal_length, 0, principal_point[0]],
         [0, focal_length, principal_point[1]],
         [0, 0, 1]], dtype = "double"
    )

    # assuming no lens distortion
    distCoeffs = np.zeros((4,1))

    # SolvePnP
    (success, rotation_vector, translation_vector) = cv2.solvePnP(model_points, image_points, camera_matrix, distCoeffs, flags=cv2.SOLVEPNP_ITERATIVE)

    return rotation_vector, translation_vector

In [5]:
import numpy as np
distCoeffs = np.zeros((4,1)) 
print(distCoeffs)

[[0.]
 [0.]
 [0.]
 [0.]]


In [6]:
def rotation_vector_to_euler_angles(rotation_vector):
    # converts a rotation vector to a rotation matrix
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    # append a zero column to make it a 3x4 matrix
    projection_matrix = np.hstack((rotation_matrix, np.zeros((3, 1))))
    # returns three-element vector containing three Euler angles of rotation in degrees.
    euler_angles = cv2.decomposeProjectionMatrix(projection_matrix)[6]
    pitch, yaw, roll = [angle for angle in euler_angles]
    
    return pitch, yaw, roll

In [7]:
def is_facing_camera(pitch, yaw, pitch_threshold=160, yaw_threshold=25):
    if abs(pitch) > pitch_threshold and abs(yaw) < yaw_threshold:
        return True
    return False

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# start up the webcam
cap = cv2.VideoCapture(1)

#initialise media pipe's face mesh and video drawing 
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# check if the webcam can be accessed
if not cap.isOpened():
    raise IOError("Cannot open webcam")

#take one frame from the webcam 
ret, frame = cap.read()

# check if the webcam is opened correctly
if ret:
    # draw the face mesh mapping onto the frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=drawing_spec,
                connection_drawing_spec=drawing_spec)
            # get rotation vector
            rotation_vector, translation_vector = get_head_pose(face_landmarks.landmark, frame.shape)
            # get pitch and yaw
            pitch, yaw, _ = rotation_vector_to_euler_angles(rotation_vector)
            print(pitch,yaw)
            # check if user is facing the camera
            if is_facing_camera(pitch, yaw):
                print("User is facing the camera.")
                cv2.putText(frame, "true", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                print("User is not facing the camera.")
                cv2.putText(frame, "false", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
    cv2.imwrite('webcam_image1.jpg', frame)
    
else:
    print("Error: No frame captured")
    

    
# close the webcam
cap.release()
# close all openCV functions
cv2.destroyAllWindows()

In [58]:
import cv2
import mediapipe as mp
import numpy as np
# initialise the codec and the video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

#initialise media pipe's face mesh and video drawing 
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(1)
# check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # convert captured frame to colour
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # apply the face mesh
        results = face_mesh.process(frame_rgb)

        # draw the face mesh mapping onto the frame
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=frame,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)

                # get rotation vector
                rotation_vector, translation_vector = get_head_pose(face_landmarks.landmark, frame.shape)
                # get pitch and yaw
                pitch, yaw, _ = rotation_vector_to_euler_angles(rotation_vector)
                # check if user is facing the camera
                if is_facing_camera(pitch, yaw):
                    cv2.putText(frame, "true", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                else:
                    cv2.putText(frame, "false", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Frame", frame)
            # write the frame on to a file using the "out" video writer
            out.write(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
# stop the recording on an interrupt
except KeyboardInterrupt:
    print("Stream stopped")
    
finally:
    # close the webcam
    cap.release()
    # close all openCV functions
    out.release()
    cv2.destroyAllWindows()

Stream stopped


In [54]:
import cv2
import numpy as np
import mediapipe as mp
import math

# Function to convert rotation vectors to euler angles
def rotation_vector_to_euler_angles(rotation_vector):
    rotation_matrix, _ = cv2.Rodrigues(rotation_vector)
    projection_matrix = np.hstack((rotation_matrix, np.zeros((3, 1))))
    euler_angles = cv2.decomposeProjectionMatrix(projection_matrix)[6]
    pitch, yaw, roll = [angle for angle in euler_angles]
    return pitch, yaw, roll

def draw_pose_line(image, nose_point, yaw, pitch, length=200):
    """ Draw a line from the nose point in the direction indicated by the yaw and pitch. """
    # Convert angles from degrees to radians
    pitch = pitch * np.pi / 180+0.07
    yaw = -yaw * np.pi / 180 + 1.6
    
    # Calculate endpoint of the line
    x = length * np.cos(pitch) * np.cos(yaw)
    y = length * np.sin(pitch)
    z = length * np.cos(pitch) * np.sin(yaw)

    # Point in the image
    end_point = (int(nose_point[0] + x), int(nose_point[1] - y))
    
    # Draw line and arrow
    cv2.line(image, (int(nose_point[0]), int(nose_point[1])), end_point, (255, 0, 0), 2)
    cv2.arrowedLine(image, (int(nose_point[0]), int(nose_point[1])), end_point, (0, 255, 0), 2, tipLength=0.3)
    
    return image

# Function to process an image
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print("Image not found:", image_path)
        return
    
    frame_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Draw landmarks
            # mp_drawing.draw_landmarks(
            #     image=image,
            #     landmark_list=face_landmarks,
            #     connections=mp_face_mesh.FACEMESH_CONTOURS,
            #     landmark_drawing_spec=drawing_spec,
            #     connection_drawing_spec=drawing_spec)
            
            # Calculate head pose
            rotation_vector, translation_vector = get_head_pose(face_landmarks.landmark, image.shape)
            pitch, yaw, _ = rotation_vector_to_euler_angles(rotation_vector)
            
            # Get nose tip location
            nose_tip = face_landmarks.landmark[1]
            nose_point = (nose_tip.x * image.shape[1], nose_tip.y * image.shape[0])
            
            # Draw the pose line
            image = draw_pose_line(image, nose_point, yaw, pitch)
    
    # Display or save the result
    cv2.imshow("Head Pose", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite('output_pose.jpg', image)

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Example usage: process a specific image
process_image('1.jpg')

C:\Users\leosh\AppData\Local\Temp\ipykernel_36368\2459741074.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  end_point = (int(nose_point[0] + x), int(nose_point[1] - y))
